In [2]:
# import libraries
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [3]:
# get dataset files

cars_listing_1_dir = '../datasets/car-listing-1'
cars_listing_2_dir = '../datasets/car-listing-2'

def get_csvs_in_dir(dir_path):
    return [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f)) and f.endswith('.csv')]


cars_1_files = get_csvs_in_dir(cars_listing_1_dir)
cars_2_files = get_csvs_in_dir(cars_listing_2_dir)


In [4]:
# peek at cars 1 datasets
df_audi = pd.read_csv(cars_1_files[0])
print(df_audi.head())

df_bmw = pd.read_csv(cars_1_files[1])
print(df_bmw.head())

df_ford = pd.read_csv(cars_1_files[2])
print(df_ford.head())

df_hyundi = pd.read_csv(cars_1_files[3])
print(df_hyundi.head())

df_merc = pd.read_csv(cars_1_files[4])
print(df_merc.head())

df_toyota = pd.read_csv(cars_1_files[5])
print(df_toyota.head())

  model  year  price transmission  mileage fuelType  tax   mpg  engineSize
0    A1  2017  12500       Manual    15735   Petrol  150  55.4         1.4
1    A6  2016  16500    Automatic    36203   Diesel   20  64.2         2.0
2    A1  2016  11000       Manual    29946   Petrol   30  55.4         1.4
3    A4  2017  16800    Automatic    25952   Diesel  145  67.3         2.0
4    A3  2019  17300       Manual     1998   Petrol  145  49.6         1.0
       model  year  price transmission  mileage fuelType  tax   mpg  \
0   5 Series  2014  11200    Automatic    67068   Diesel  125  57.6   
1   6 Series  2018  27000    Automatic    14827   Petrol  145  42.8   
2   5 Series  2016  16000    Automatic    62794   Diesel  160  51.4   
3   1 Series  2017  12750    Automatic    26676   Diesel  145  72.4   
4   7 Series  2014  14500    Automatic    39554   Diesel  160  50.4   

   engineSize  
0         2.0  
1         2.0  
2         3.0  
3         1.5  
4         3.0  
     model  year  price tra

In [5]:
# add column for brand of car
df_audi['brand']   = 'audi'
df_bmw['brand']    = 'bmw'
df_ford['brand']   = 'ford'
df_hyundi['brand'] = 'hyundi'
df_merc['brand']   = 'merc'
df_toyota['brand'] = 'toyota'

# concatenate all dataframes together
df_cars_1 = pd.concat([df_audi, df_bmw, df_ford, df_hyundi, df_merc, df_toyota])

# change column order to something that allows us to split it easier later on
df_cars_1 = df_cars_1[['brand', 'model', 'transmission', 'fuelType','year', 'mileage', 'tax', 'mpg', 'engineSize', 'price']]
cars_1_y = df_cars_1.pop('price').to_numpy()
cars_1_X = df_cars_1.to_numpy()


# Encode Data
Using OneHotEncoder

In [6]:
# temporarily separate categorical cols from numerical
num_cols = cars_1_X[:,4:]
cat_cols = cars_1_X[:, :4]

# One-Hot Encode string values
enc = OneHotEncoder(sparse=False)
cat_cols_enc = enc.fit_transform(cat_cols)

cars_1_X_enc = np.hstack((cat_cols_enc, num_cols)).astype(np.float32)
print(cars_1_X_enc.shape)
print(cars_1_X_enc[:10])


(64131, 154)
[[  1.    0.    0.  ... 150.   55.4   1.4]
 [  1.    0.    0.  ...  20.   64.2   2. ]
 [  1.    0.    0.  ...  30.   55.4   1.4]
 ...
 [  1.    0.    0.  ...  20.   70.6   2. ]
 [  1.    0.    0.  ...  20.   60.1   1.4]
 [  1.    0.    0.  ...  30.   55.4   1.4]]


# Train-Test Split

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    cars_1_X_enc, cars_1_y, test_size=0.2, shuffle=True)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_test shape: {y_test.shape}")

X_train shape: (51304, 154)
y_train shape: (51304,)
X_test shape: (12827, 154)
X_test shape: (12827,)


# Loss Functions

## 1. MAE: 
- It is not very sensitive to outliers in comparison to MSE since it doesn't punish huge errors. 
- It is usually used when the performance is measured on continuous variable data. 
- It gives a linear value, which averages the weighted individual differences equally. 
- The lower the value, better is the model's performance.

## 2. MSE: 
- It is one of the most commonly used metrics, but least useful when a single bad prediction 
- would ruin the entire model's predicting abilities, i.e when the dataset contains a lot of noise. 
- It is most useful when the dataset contains outliers, or unexpected values(too high or too low values).

## 3. RMSE: 
- In RMSE, the errors are squared before they are averaged.
- This basically implies that RMSE assigns a higher weight to larger errors. 
- This indicates that RMSE is much more useful when large errors are present and they drastically affect the model's performance. 
- It avoids taking the absolute value of the error and this trait is useful in many mathematical calculations.
- In this metric also, the lower the value, better is the performance of the model.

In [8]:
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

# TODO - try other loss functions and determine best to use


def BaselineModel(optimizer='adam', loss='mean_squared_error',
                  activation='relu', output_activation='linear',
                  kernel_initializer='glorot_uniform', bias_initializer='zeros',
                  input_neurons=150, hidden_neurons=200, num_hidden_layers=2):

    model = Sequential()
    
    # Define INPUT layer
    model.add(Dense(input_neurons, input_dim=X_train.shape[1],
                    activation=activation,
                    kernel_initializer=kernel_initializer, 
                    bias_initializer=bias_initializer,
                    name='layer_input'))
    
    for i in range(num_hidden_layers):
        model.add(Dense(hidden_neurons, activation=activation,
                        kernel_initializer=kernel_initializer, 
                        bias_initializer=bias_initializer,
                        name=f'layer_hidden_{i}'))
    
    # Define OUTPUT layer
    model.add(Dense(1, activation=output_activation,
                    kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer, 
                    name='layer_output'))

    # Compile model
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    # Print out model summary
    print(model.summary())
    return model


# Define checkpoint callback for model saving
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
log_dir = 'tensorboard-logs' 

checkpoint = ModelCheckpoint(f'models/{checkpoint_name}', monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='auto')
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [checkpoint, early_stopping, tensorboard]


In [9]:
from keras.callbacks import TensorBoard
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# Run with default parameters

coarse_grid = [
    {
        'batch_size': [10, 50, 100, 200, 400],
        'epochs': [25, 50, 100],
        'optimizer': ['Adam'],
        'activation': ['relu'],
        'num_hidden_layers': [1, 2, 5],
        'hidden_neurons': [50, 100, 200, 500]
    }
]

cross_val = 5
scores = ['precision', 'recall']
result = {}
for score in scores:
    print('-' * 50)
    print(
        f"# Tuning hyper-parameters for {score} with {cross_val}-fold cross-validation")
    print()

    # Employ GridSearch using the cross_val variable on the param grid provided
    clf = GridSearchCV(
        KerasRegressor(build_fn=BaselineModel, batch_size=100, epochs=100, verbose=2),
        coarse_grid,
        scoring='%s_macro' % score,
        # cv=cross_val,
        n_jobs=1,
        verbose=2
    )
    # Fit the model on the training labels and outputs
    
    clf.fit(X_train, y_train,
            shuffle=True)
    
    # clf.fit(X_train, y_train,
    #         callbacks=[early_stopping],
    #         verbose=2, shuffle=True,
    #         workers=2, use_multiprocessing=True)
    

    print("# Best parameters set found on development set:")
    print(f'\t{clf.best_params_}')
    print()
    result[score] = clf.best_params_
    print("# Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("\t - %0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
    print()

    print("# Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()



--------------------------------------------------
# Tuning hyper-parameters for precision with 5-fold cross-validation

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [ ]:

# history = model_0.fit(X_train, y_train, validation_split=0.2,
#                       batch_size=50, epochs=100,
#                         callbacks=callbacks_list,
#                         verbose=1, shuffle=True,
#                         workers=8, use_multiprocessing=True)

# Print the precision and recall of the model.
# y_pred = model_0.predict(X_test)

# print("MAE", mean_absolute_error(y_test, y_pred))


# print(history[1].params)

# loss_train = history['train_loss']
# loss_val = history['val_loss']
# epochs = range(1, 100)
# plt.plot(epochs, loss_train, 'g', label='Training loss')
# plt.plot(epochs, loss_val, 'b', label='validation loss')
# plt.title('Training and Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# loss_train = history['acc']
# loss_val = history['val_acc']
# epochs = range(1, 100)
# plt.plot(epochs, loss_train, 'g', label='Training accuracy')
# plt.plot(epochs, loss_val, 'b', label='validation accuracy')
# plt.title('Training and Validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()


IndexError: tuple index out of range